In [1]:
import os
import numpy as np
from scipy import integrate
from astropy.modeling.blackbody import blackbody_lambda
from astropy import units as u

In [2]:
# Ratio of continuum energy to bolometric energy, per Osten & Wolk (2015).
Econt_Ebol = 0.6

In [3]:
# Calculate continuum emission of 10,000 K blackbody.
cont = np.arange(1400, 10000, 1) * u.AA
bb_cont = blackbody_lambda(in_x=cont, temperature=10000 * u.Kelvin)

In [4]:
# This handy little function will read in a filter profile file from the SVO Filter Service so we can get the
# approximate min. and max. wavelength of a filter.
def read_svo_file(ifile):
    with open(ifile, 'r') as inputf:
        flines = inputf.readlines()
    # Parse to get first wavelength with non-zero transmission in the file.
    start_wl = None
    index = 0
    while start_wl is None:
        if index < len(flines):
            if float(flines[index].strip().split()[1]) > 5.0:
                start_wl = float(flines[index].strip().split()[0])
        else:
            break
        index += 1
    # Parse to get last wavelength with non-zero transmission in the file.
    end_wl = None
    index = -1
    while end_wl is None:
        if index > -1*len(flines):
            if float(flines[index].strip().split()[1]) > 5.0:
                end_wl = float(flines[index].strip().split()[0])
        else:
            break
        index -= 1
    return (start_wl, end_wl)

In [5]:
# Reproduce the NUV calculation from Brasseur et al. (2019) of the fraction of the bolometric luminosity contained
# within the GALEX NUV band.  Instead of using the filter wavelength range cited in their paper, we rely on the SVO
# filter service and consider the start and end wavelengths to be those that have at least 5% transmission
#(we note the results are essentially identical and well within the uncertainties in the approximate calculation).
nuv_filter_file = "GALEX_GALEX.NUV.dat.txt"
if os.path.isfile(nuv_filter_file):
    start_wl, end_wl = read_svo_file(nuv_filter_file)
else:
    raise IOError("Missing SVO lookup file: " + nuv_filter_file + " from repository.")
nuv = np.arange(start_wl, end_wl, 1) * u.AA
bb_nuv = blackbody_lambda(in_x=nuv, temperature=10000 * u.Kelvin)
ENUV_Econt = np.trapz(x=nuv, y=bb_nuv) / np.trapz(x=cont, y=bb_cont)
print('p_bol (NUV) = ', float(ENUV_Econt) * Econt_Ebol)
print('NOTE: p_bol (NUV) using 1771 and 2831 Angstroms as start/stop like in Brasseur et al. 2019:'
          ' 0.13152561432613658')

p_bol (NUV) =  0.13331626928915136
NOTE: p_bol (NUV) using 1771 and 2831 Angstroms as start/stop like in Brasseur et al. 2019: 0.13152561432613658


In [6]:
# Compute the fraction of the bolometric luminosity contained within the GALEX FUV band.
fuv_filter_file = "GALEX_GALEX.FUV.dat.txt"
if os.path.isfile(fuv_filter_file):
    start_wl, end_wl = read_svo_file(fuv_filter_file)
else:
    raise IOError("Missing SVO lookup file: " + fuv_filter_file + " from repository.")
fuv = np.arange(start_wl, end_wl, 1) * u.AA
bb_fuv = blackbody_lambda(in_x=fuv, temperature=10000 * u.Kelvin)
EFUV_Econt = np.trapz(x=fuv, y=bb_fuv) / np.trapz(x=cont, y=bb_cont)
print('p_bol (FUV) = ', float(EFUV_Econt) * Econt_Ebol)

p_bol (FUV) =  0.020583537488146248


Compute the p_bol fraction for Johnson U-band, Kepler, and TESS bandpasses.